## Export Blue Carbon tiffs

* #### **Step 1:** Take Blue carbon Asset from GEE and export to the cloud storage as tiffs
* #### **Step 2:** Take exported tiffs and unify into one single file and reupload to GCS

Requires autentication into Google earth Engine and into Google Cloud Storage. This notebook uses credentials stored in a file `service_account.json`

### Step 1: Export Asset (GEE) to tiff (GCS)

In [ ]:
import ee
import os

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
region = ee.Geometry.Polygon([-180, 40, 0,40,180, 40,180,-40,0,-40,-180,-40])
coll = ee.ImageCollection("projects/global-mangrove-watch/mangrove-properties/mangrove_total_co2e_1996--2016").select('total_co2e')
collectionList = coll.toList(coll.size())
collectionSize = collectionList.size().getInfo()

In [ ]:
i= 0
img = ee.Image(collectionList.get(i))
img_name = ee.String(img.get("system:id")).split("/").get(-1).getInfo()
year = img_name[-4:]

params = {
'image':img,
'description':'toc_'+year,
'bucket':'mangrove_atlas',
'fileNamePrefix':'ee_export_tiffs/total_organic_carbon/toc_'+year+'/toc_'+year, 
'fileFormat':'GeoTIFF',
'region': region.getInfo()['coordinates'], 
'skipEmptyTiles': False,
'maxPixels': 1e12,
'scale':30
}

task = ee.batch.Export.image.toCloudStorage(**params)
task.start()

In [ ]:
for i in range(1,collectionSize):
    img = ee.Image(collectionList.get(i))
    img_name = ee.String(img.get("system:id")).split("/").get(-1).getInfo()
    year = img_name[-4:]

    params = {
    'image':img,
    'description':'toc_'+year,
    'bucket':'mangrove_atlas',
    'fileNamePrefix':'ee_export_tiffs/total_organic_carbon/toc_'+year+'/toc_'+year, 
    'fileFormat':'GeoTIFF',
    'region': region.getInfo()['coordinates'], 
    'skipEmptyTiles': False,
    'maxPixels': 1e12,
    'scale': 30,
    }

    task = ee.batch.Export.image.toCloudStorage(**params)
    task.start()

In [ ]:
# Check task status
!earthengine task list --status RUNNING

In [ ]:
!earthengine task list

### Step 2: Download tiffs (from GCS), Unify into 1 file (gdal) and upload (back to GCS)

In [ ]:
path_to_gcs_credentials = '../../data/service_account.json'

In [ ]:
from google.cloud import storage

def upload_blob(bucket_name, source_file_name, destination_blob_name, path_to_gcs_credentials):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client.from_service_account_json(path_to_gcs_credentials)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )
#https://stackoverflow.com/questions/49748910/python-download-entire-directory-from-google-cloud-storage    
def download_blob(bucket_name, prefix, dest_folder,path_to_gcs_credentials):
    """Downloads a file to the bucket."""
     # bucket_name = 'your-bucket-name'
     # prefix = 'your-bucket-directory/' or 'your-bucket-directory/and-file-name'
     # dest_folder = 'your-local-directory/'

    storage_client = storage.Client.from_service_account_json(path_to_gcs_credentials)

    # Retrieve all blobs with a prefix matching the folder
    bucket=storage_client.get_bucket(bucket_name)
    print(bucket)
    blobs=list(bucket.list_blobs(prefix=folder))

    for blob in blobs:
        try: 
            if(not blob.name.endswith("/")):
                file_split = blob.name.split("/")
                blob.download_to_filename(f"{dest_folder}{file_split[-1]}")
        except OSError as err:
            print("OS error: {0}".format(err))
            print(file_split[-1])
            continue
    

In [ ]:
y=1996
bucket_name = 'mangrove_atlas'

## example for a set of files with the prefix toc_19960000023296
## to export complete folder, stop path at the previous folder level
prefix=f'ee_export_tiffs/total_organic_carbon/toc_{y}/toc_19960000023296'
dest_folder_tiffs = '../../data/toc_temporary/
dest_folder_mosaic = "../../data/gmw_blue_carbon/"
os.makedirs(dest_folder_tiffs, exist_ok=True)
os.makedirs(dest_folder_mosaic, exist_ok=True)

In [ ]:
%%time
#for y in [1996, 2007, 2008, 2009, 2010, 2015, 2016, 2017, 2018, 2019, 2020]:
for y in [1996]: ## test one year
    #prefix=f'ee_export_tiffs/total_organic_carbon/toc_{y}/toc_19960000023296'
    #download_blob(bucket_name,prefix,dest_folder_tiffs)
    #print("\nBuilding VRT\n")
    #!gdalbuildvrt {dest_folder_mosaic}/gmw_toc_{y}.vrt {dest_folder_tiffs}/*.tif
    print("\nBuilding GeoTIFF\n")
    !gdal_translate -co "BLOCKXSIZE=512" -co "BLOCKYSIZE=512" -co "TILED=YES" -co "BIGTIFF=YES" -co "COMPRESS=DEFLATE" {dest_folder_mosaic}/gmw_toc_{y}.vrt {dest_folder_mosaic}/gmw_toc_{y}.tif
    print("\nCopying files to GCS\n")
    upload_blob('mangrove_atlas', f'{dest_folder_mosaic}/gmw_toc_{y}.tif', f'ee_export_tiffs/total_organic_carbon/v2/toc_{y}.tif')

### Note (Feb 9th 2022)
This test was made with a subset of files, this must be done with the complete folder to check that the final file has the global extent. The download of tiffs and its processing takes an important amount of disk space.